In [ ]:
import scipy as sp
import numpy as np
import scipy.sparse as sparse
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})
%matplotlib inline

import stencil

# Define the $h$-norm

$$
\|r\|_{h} = h \|r\|_{2}
$$
for any vector $r$

In [ ]:
def hnorm(r):
    """define ||r||_h = h ||r||_2"""
    n = len(r)
    h = 1.0 / (n+1)
    hrnorm = h * np.linalg.norm(r)
    return hrnorm

# Multigrid Operators

Form relaxation, interpolation (based on 1D), and the matrix operator $A$.

In [ ]:
def relax(A, u, f, nu):
    n = A.shape[0]
    unew = u.copy()
    DE = sparse.tril(A, 0).tocsc()
    
    for i in range(nu):
        unew += sla.spsolve(DE, f - A * unew, permc_spec='NATURAL')

    return unew

def interpolation1d(nc, nf):
    d = np.repeat([[1, 2, 1]], nc, axis=0).T
    I = np.zeros((3,nc),dtype=int)
    for i in range(nc):
        I[:,i] = [2*i, 2*i+1, 2*i+2]
    J = np.repeat([np.arange(nc)], 3, axis=0)
    P = sparse.coo_matrix(
        (d.ravel(), (I.ravel(), J.ravel()))
        ).tocsr()
    return 0.5 * P

def create_operator(n, sten):
    """
    Create a 2D operator from a stencil.
    """
    A = stencil.stencil_grid(sten, (n, n), format='csr')
    return A

# Two-level Method

An example two-level method:

- pre-semooth
- restrict
- coarse solve
- interpolate
- post-smooth

In [ ]:
def twolevel(A, P, A1, u0, f0, nu):
    u0 = relax(A, u0, f0, nu) # pre-smooth
    f1 = P.T * (f0 - A * u0)  # restrict

    u1 = sla.spsolve(A1, f1)  # coarse solve

    u0 = u0 + P * u1          # interpolate
    u0 = relax(A, u0, f0, nu) # post-smooth
    return u0

# Test Problem

This runs a problem with $f=0$.

In [ ]:
# Problem setup
k = 9
n = 2**k - 1
nc = 2**(k-1) - 1
sten = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
A = (n+1)**2 * create_operator(n, sten)
f = np.zeros(n*n)
u = np.random.rand(n*n)

# Multigrid Setup
P1d = interpolation1d(nc, n)
P = sparse.kron(P1d, P1d).tocsr()
A1 = P.T * A * P

# Multigrid cycling
res = [hnorm(f - A * u)]
for i in range(10):
    u = twolevel(A, P, A1, u, f, 2)
    res.append(hnorm(f - A * u))

# Look at the residuals
res = np.array(res)
print(res[1:] / res[:-1])

In [ ]:
res

In [ ]:
plt.semilogy(res, label='residual')
plt.legend()